In [ ]:
import numpy as np

def interp2D(spots, times, vols, spot, time):
    """
    Bilinear interpolation for local volatility.
    This function interpolates the volatility surface at a given spot and time.
    """
    # Find indices for interpolation
    spot_idx = np.searchsorted(spots, spot) - 1
    time_idx = np.searchsorted(times, time) - 1

    # Clamp indices to valid range
    spot_idx = np.clip(spot_idx, 0, len(spots) - 2)
    time_idx = np.clip(time_idx, 0, len(times) - 2)

    # Get bounding points for interpolation
    x0, x1 = spots[spot_idx], spots[spot_idx + 1]
    y0, y1 = times[time_idx], times[time_idx + 1]
    z00 = vols[spot_idx, time_idx]
    z01 = vols[spot_idx, time_idx + 1]
    z10 = vols[spot_idx + 1, time_idx]
    z11 = vols[spot_idx + 1, time_idx + 1]

    # Bilinear interpolation
    tx = (spot - x0) / (x1 - x0) if x1 > x0 else 0.0
    ty = (time - y0) / (y1 - y0) if y1 > y0 else 0.0
    vol = (1 - tx) * (1 - ty) * z00 + tx * (1 - ty) * z10 + (1 - tx) * ty * z01 + tx * ty * z11
    return vol

def toy_dupire_barrier_mc(S0, spots, times, vols, maturity, strike, barrier, Np, Nt, epsilon, rng):
    """
    Monte Carlo simulation for barrier option pricing in Dupire's model.
    
    Parameters:
    - S0: Initial spot price
    - spots: Array of spot levels for local volatility surface
    - times: Array of time levels for local volatility surface
    - vols: Local volatility surface (2D array)
    - maturity: Option maturity
    - strike: Option strike price
    - barrier: Barrier level
    - Np: Number of Monte Carlo paths
    - Nt: Number of time steps
    - epsilon: Smoothing parameter for barrier logic
    - rng: Random number generator, should have a `normal` method for generating Gaussian increments

    Returns:
    - Price of the barrier option
    """
    result = 0.0
    dt = maturity / Nt
    sdt = np.sqrt(dt)

    for i in range(Np):
        # Generate Gaussian increments
        gaussian_increments = rng.normal(0, 1, Nt)

        # Initialize path variables
        spot = S0
        time = 0.0
        alive = 1.0  # Path survival indicator

        for j in range(Nt):
            # Interpolate local volatility
            vol = interp2D(spots, times, vols, spot, time)
            time += dt

            # Simulate next step of the path
            spot *= np.exp(-0.5 * vol**2 * dt + vol * sdt * gaussian_increments[j])

            # Monitor barrier
            if spot > barrier + epsilon:
                alive = 0.0  # Definitely dead
                break
            elif spot < barrier - epsilon:
                continue  # Definitely alive
            else:
                # Interpolate survival probability
                alive *= 1.0 - (spot - (barrier - epsilon)) / (2 * epsilon)

        # Payoff calculation
        if spot > strike:
            result += alive * (spot - strike)  # Payoff on surviving notional

    return result / Np

# Example usage
if __name__ == "__main__":
    np.random.seed(42)  # Fix random seed for reproducibility
    S0 = 100.0
    spots = np.linspace(80, 120, 10)
    times = np.linspace(0, 2, 5)
    vols = np.random.uniform(0.1, 0.3, (len(spots), len(times)))  # Example local volatility surface
    maturity = 1.0
    strike = 100.0
    barrier = 110.0
    Np = 10000
    Nt = 50
    epsilon = 0.01
    rng = np.random.default_rng()  # Random number generator

    price = toy_dupire_barrier_mc(S0, spots, times, vols, maturity, strike, barrier, Np, Nt, epsilon, rng)
    print(f"Barrier Option Price: {price:.4f}")
